In [0]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt


**Definition - One sided Z-transform**

The one sided z-transform is defined as 

$$ X(z) = Z[x[n]] = \sum_{n=0}^{\infin} x[n]z^{-n} $$

The z-transform is used to find exponentially increasing or decreasing oscillations in a signal.

This is used in discrete systems to evaluate their stability, frequency response and many other properties.

The "n" in the z-transform is the sample number

**Example - Z-transform of a simple signal**

Consider the signal \\(x=[4,-1, 6,-3, 5]\\)

Taking the z-transform, for the DC component \\(z=1\\) yields

$$ X(1)=4*1 + (-1) * 1 + 6 * 1 + (-3) * 1 + 5 * 1 = 11$$

This makes sense, since the mean value is above zero for the signal, meaning it has a DC component.

For the signal \\(x=[4,-1, 6,-3, -6]\\) we would have the z-transform 

$$ X(1)=4*1 + (-1) * 1 + 6 * 1 + (-3) * 1 + (-6) * 1 = 0$$

Here the mean value is zero, and the z-transform correctly tells us that there is no correlation with the DC component

**Definition - Z**

The number \\(z\\) in the z-transform should be able to represent both exponentially increasing and decreasing oscillations, which means that the variable \\(z\\) is a complex number.

$$ z = r e^{j \omega} $$

Here \\(r\\) is a real number representing the magnitude of the signal and \\(\omega\\) is the angular frequency in radians/sample.

When \\(z\\) is raised to the power of \\(-n\\) something interesting happens. It can be written as 

$$ z^{-n} = (r e^{j \omega })^{-n} = r^{-n} e^{-j \omega n} $$


Here we see that the magnitude will decrease with \\(n\\) if \\(r>1\\). If \\(r=1\\) then the magnitude will be unchanged.


**Definition - Normalized frequency**

If we have a frequency of \\(\omega=\frac{\pi}{4}[radians]/[sample]\\) and a complete cycle of a sinusoid is \\(2 \pi [radians]\\) we must have 8 samples in a cycle.

$$ 2 \pi[radians]/[cycle] * \frac{4}{\pi} [sample]/[radians] = 8 [samples]/[cycles] $$

This means that the period is \\(8T\\) where \\(T\\) is the sampling interval and \\(f_s=\frac{1}{T}\\) is the sampling interval. The frequency of the sinusoid can then be expressed as \\(f_{sin}=\frac{1}{8T}[Hz]=\frac{1}{8}\frac{1}{T}[Hz]=\frac{f_s}{8}[Hz]\\)

Here we see that the frequency of the sinusoid depends on \\(f_s\\) and the normalized freqeuncy is defined as.

$$ f_{norm} = \frac{f}{f_s}$$

Rewriting the frequency of the sinusoid we have 

$$ f_{norm} [cycles]/[sample] = \frac{f[cycles]/[second]}{f_s [samples]/[second]}= \frac{1}{8} [cycles]/[sample] $$

The same thing can be done for angular frequency where \\(\omega_{norm} = \frac{\omega}{f_S} [radians][sample]\\)

In [0]:
omega = np.pi/4
x = np.arange(16)
s = np.cos(x*omega)

plt.figure()
plt.plot(x, s, "-*", label="Cosine")
plt.title("Two cycles of a cosine")

In [0]:
### The purpose of this example is to illustrate the z-transform using three different values of r and omega

def plot_z_transform(r, omega, N):

    n = np.arange(N)

    y = 1.0/(r**n) * np.exp(-1j * omega * n)

    plt.figure()
    plt.plot(n, np.real(y), "-*", label="Real part")
    plt.plot(n, np.imag(y), "-*", label="Imaginary part")
    plt.title(f"r = {r}, omega = {omega:.3f}, N = {N}")
    plt.xlabel("Samples")
    plt.ylabel("Magnitude")
    plt.legend()

plot_z_transform(1.1, np.pi/4, 20)
plot_z_transform(1.0, np.pi/8, 20)
plot_z_transform(0.9, np.pi/5, 20)



**Example - Z-transform of as system impulse response**

Given a system described by the function \\(y_n=0.2x_n+0.4x_{n-1}\\) we can calculate the impulse response as 

$$ h = [0.2, 0.4, 0, 0, 0] $$

The z-transform of this impulse response defined as

$$ H(z) = \sum_{n=0}^\infin h[n] x^{-n} $$

And can be calculated to

$$ H(z) = 0.2z^{-0}+0.4z^{-1}$$

By plotting different values of z into this equation, we can visualize the repsonse of the system


In [0]:
## This example visualized the response of a system, using the impulse response and the z-transform
# It can be seen that the system has a pole(positive infinite value ) at the origin and a zero(negative infinite value) at -2+0j

def z_transform(h , realv, imagv, limit=None):

    result = np.zeros_like(realv) + 0.0+0.0j

    for n, h_n in enumerate(h):
        if limit is not None and np.any(result > limit):
            break
        result += h_n * (realv+1j*imagv)**(-n)
        
    return result

points = 100
real = np.linspace(-2, 2, points)
imag = np.linspace(-2, 2, points)
realv, imagv = np.meshgrid(real, imag)

h = [0.2, 0.4]

h_z = z_transform(h, realv, imagv)

plt.contourf(realv, imagv, 20*np.log10(np.abs(h_z)))
plt.axis('scaled')
plt.colorbar()
plt.show()




**Definition - Region of Convergence(ROC)**

Consider the system 

$$ y_n = x_n + 0.5 y_{n-1} $$

Here we have a feedback loop in the \\(0.5 y_{n-1}\\) term. This term will cause the impulse response of a system to be inifinite. 

_ROC Property 1_

The impulse response of a system is absolutely summable

$$ |X(z)| < \infin $$

_ROC Property 2_

There can be no poles in the ROC, since by definition a, the response of the system at a pole is infinite.

_ROC Property 3_

For a system with inifinte impulse response, there will be a cicrcle around the origin where the impulse response is NOT absolutely summable.

_ROC Property 4_

If the regions of convergence does not include the unit circle, the system is unstable.

In [0]:
## This example illustrates the region of convergence.
# Here is is seen that there is a region where the impulse repsonse does not converge.

# Generate the impulse response
x = np.zeros(1000)
x[0] = 1
h = signal.lfilter([1], [1, -0.5], x)

# Plot the z transform and the region of convergence
points = 100
real = np.linspace(-2, 2, points)
imag = np.linspace(-2, 2, points)
realv, imagv = np.meshgrid(real, imag)

h_z = z_transform(h, realv, imagv)

plt.figure()
plt.contourf(realv, imagv, 20*np.log10(np.abs(h_z)))
plt.axis('scaled')
plt.colorbar()
plt.show()



**Time shifting property**

Shifting a signal \\(x[n]\\) by some constant \\(k\\) yields a multiplication by \\(z^{-k}\\) in the z domain.

$$ X(z)z^{-k} = \sum_{n=0}^\infin x[n-k] z^{-n} $$

**Time shifting property example**

Consider the signal $$x = [3, -1, 2, 1, 0, 0]$$

And the delayed version \\(w[n-2]\\)

$$w = [0, 0, 3, -1, 2, 1]$$

Taking the z transform of x yields

$$X = [3z^{-0}-1z^{-1}+2z^{-2}+1z^{-3}] $$

Taking the z transform of w yields

$$X = [3z^{-2}-1z^{-3}+2z^{-4}+1z^{-5}] = [3z^{-0}z^{-2}-1z^{-1}z^{-2}+2z^{-2}z^{-2}+1z^{-3}z^{-2}] $$

Here it is seen that the z transform of w is the same as the tranformed version of x, only multiplied by \\(z^{-2}\\)

This property is only valid if the transform converges!


**Linear property**

The z transform of the sum of two signals, each scaled by a constant, is the same as the sum of their z transform scaled by the same constant. 

If 

$$ X(z) = \sum_{n=0}^\infin x[n] z^{-n} $$

And

$$ Y(z) = \sum_{n=0}^\infin y[n] z^{-n} $$

Then the following is true 

$$ c_1 x(z) + c_2 Y(z) =  \sum_{n=0}^\infin (c_1x[n] + c_2y[n]) z^{-n} $$

**Linear propery example**

Consider the signals $$x = [3, -1, 2, 1]$$ $$w = [1, 1, 1, 1]$$ $$y = x+w = [4, 0, 3, 2]$$

And their z transforms

$$ X = [3z^{-0}-1z^{-1}+2z^{-2}+1z^{-3}] $$ 
$$ W = [1z^{-0}+1z^{-1}+1z^{-2}+1z^{-3}] $$ 
$$ Y = [4z^{-0}+0z^{-1}+3z^{-2}+2z^{-3}] $$

Here it is clearly seen that \\(Y = X + W\\)

This property is only valid if the transform converges!

**Analysis of difference equations**

The time shifting property and the linearity property can be used to analyze difference equations using the z transform

Consider the difference equation

$$ y[n] = x[n] - 0.2 x[n-1] - 0.8 x[n-2] + 0.05 y[n-1] + 0.03 y[n-2]$$

The z transform can be calculated using the two properties

$$ Y(z) = X(z) - 0.2 X(z)z^{-1} - 0.8 X(z)z^{-2} + 0.05 Y(z)z^{-1} + 0.03 Y(z)z^{-2}$$

To find the transfer function \\(H(z)=\frac{Y(z)}{X(z)}\\) we rearrange and  factor out \\(X(z)\\) and \\(Y(z)\\).

$$ Y(z)(1 - 0.05 z^{-1} - 0.03 z^{-2})= X(z)(1 - 0.2 z^{-1} - 0.8 z^{-2})$$

$$ \frac{Y(z)}{X(z)} = \frac{(1 - 0.2 z^{-1} - 0.8 z^{-2})}{(1 - 0.05 z^{-1} - 0.03 z^{-2})} $$

Finding the poles of the transfer function is done by either setting the numerator equal to zero and solving for z or simply facoring the expressions.

$$ \frac{Y(z)}{X(z)} = \frac{(1 - 0.2 z^{-1} - 0.8 z^{-2})}{(1 - 0.05 z^{-1} - 0.03 z^{-2})} = \frac{(z - 1) (z + 0.8)}{(z - 0.2) (z + 0.15)}$$

This yields zeros at \\(z=1\\) and \\(z=-0.8\\) and poles at \\(z=0.2\\) and \\(z=-0.15\\)





In [0]:
## This example illustrates the analysis of difference equations

# Generate the impulse response
x = np.zeros(1000)
x[0] = 1
h = signal.lfilter([1, -0.2, -0.8], [1, -0.05, -0.03], x)

# Plot the z transform and the region of convergence
points = 100
real = np.linspace(-1.2, 1.2, points)
imag = np.linspace(-1, 1, points)
realv, imagv = np.meshgrid(real, imag)

h_z = z_transform(h, realv, imagv)

plt.figure()
plt.contourf(realv, imagv, 20*np.log10(np.abs(h_z)))
plt.axis('scaled')
plt.colorbar()
plt.show()